In [6]:
from googlesearch import search
import pandas as pd
import numpy as np 
import time
import urllib3
from swiftshadow.classes import Proxy
from itertools import cycle

In [7]:
to_scrap = pd.read_excel('data/to_scrap.xlsx')


In [16]:
#Extract companies with nan as website
to_scrap.loc[to_scrap['Site internet de la cible'].isna(), 'Société cible ou acteur']

114                                 PIXIKA (GOO BUSINESS)
132                              SMART HOME INTERNATIONAL
202                                      SPORTSCAPE GROUP
203                                                DELZIA
213                         STRATEGIE MEDIA CONSEIL (SMC)
291                                    CORE DEVELOPPEMENT
329                                          SOLWARE LIFE
370                                    ADDINSOFT (XLSTAT)
442                                        COPIE CONFORME
461                                                 INLOG
474                                                 AEROW
493                               EKSAÉ (EX CEGID PUBLIC)
684                    ACTIVITÉ TELECOM SYSTEMS DE NEXANS
894                                             MILLENIUM
956     C-BEYOND (B-HIVE, BTM CONSULTANTS, ENERJIA, AL...
976                                              EVARISTE
986                   SOCIÉTÉ NAZAIRIENNE D'INTÉRIM (SNI)
1028          

In [8]:
#Example de comment on utilise
list = [k for k in search('49-3', num_results= 5, lang = 'fr')]
list

['https://www.vie-publique.fr/fiches/19494-le-recours-larticle-493-de-la-constitution',
 'https://www.legifrance.gouv.fr/loda/article_lc/LEGIARTI000006527529/1999-07-09',
 'https://fr.wikipedia.org/wiki/Article_49_alin%C3%A9a_3_de_la_Constitution_de_la_Cinqui%C3%A8me_R%C3%A9publique_fran%C3%A7aise',
 'https://www.midilibre.fr/2024/12/02/utilisation-du-49-3-motion-de-censure-a-quoi-sattendre-dans-les-prochaines-heures-et-les-jours-qui-suivent-12361561.php',
 'https://www.mediapart.fr/journal/politique/021224/michel-barnier-choisit-le-49-3-et-la-chute-de-son-gouvernement']

In [37]:
proxies_list = pd.read_excel('data/Proxies.xlsx').iloc[:,0].to_list()
proxies_cycle = cycle(proxies_list)  # Create a cyclic iterator for proxies

In [38]:
proxies_list

['http://114.130.153.122:58080',
 'http://52.79.107.158:8080',
 'http://78.128.60.112:48729',
 'http://192.252.215.5:16137',
 'http://14.241.182.44:5678',
 'http://5.9.43.92:30322',
 'http://178.20.45.29:80',
 'http://211.118.30.69:1080',
 'http://142.93.113.81:7497',
 'http://47.88.59.79:82',
 'http://182.184.33.34:1524',
 'http://91.185.236.239:4145',
 'http://177.23.176.58:8080',
 'http://185.200.38.14:8080',
 'http://45.235.123.45:999',
 'http://103.155.184.122:1088',
 'http://212.110.188.202:34409',
 'http://66.29.128.241:50873',
 'http://103.79.152.203:5678',
 'http://206.189.130.107:8080',
 'http://27.72.145.184:5678',
 'http://83.228.101.139:60606',
 'http://50.174.7.156:80',
 'http://190.184.144.222:5678',
 'http://181.212.41.171:999',
 'http://103.151.41.7:80',
 'http://37.98.218.137:5678',
 'http://64.137.108.197:5790',
 'http://185.32.45.201:4153',
 'http://195.175.29.38:9090',
 'http://156.228.89.227:3128',
 'http://67.22.28.62:8080',
 'http://182.72.203.255:80',
 'http://

In [9]:
# Disabling warnings about ssl certificated because we will be making requests without ssl (cause some website have expired ssl certificates we need to ignore)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [22]:



swift = Proxy(protocol='http', maxProxies= 100, autoRotate=True, cachePeriod= 60, countries = ['FR', 'DE', 'ES', 'BE', 'IT', 'GB', "NL", "PL"]) 

# Function to fetch the first URL for a company using the search function
def fetch_first_url(company_name):
    while True:  # Keep trying until a valid proxy succeeds
        
        proxy = swift.proxy()  # Get the next proxy
        proxy_string = "".join([proxy[1], "//", proxy[0]])
        try:
            print(f"Searching for '{company_name}' using proxy: {proxy_string}")
            # Perform the search, fetching only 1 result
            results = search(
                term=company_name + ' official company website -site:instagram.com -site:linkedin.com -site:annuaire-entreprises.data.gouv.fr', 
                num_results=1, 
                proxy=proxy_string, 
                sleep_interval=0, ssl_verify = False, timeout = 0.1
            )
            # Return the first result URL
            return next(results, None)
        except Exception as e:
            print(f"Proxy {proxy} failed with error: {e}. Trying the next proxy...")



2024-12-03 22:35:31,041 - swiftshadow [INFO]:Loaded proxies from cache


In [23]:
#On extrait tous les sites web 
series_websites = to_scrap.loc[:,'Société cible ou acteur'].apply(lambda x : fetch_first_url(x))

Searching for 'AETHER FINANCIAL SERVICES' using proxy: http//51.159.159.73:80
Proxy ['51.159.159.73:80', 'http'] failed with error: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=AETHER+FINANCIAL+SERVICES+official+company+website+-site%3Ainstagram.com+-site%3Alinkedin.com+-site%3Aannuaire-entreprises.data.gouv.fr&num=3&hl=en&start=0&safe=active (Caused by ProxyError('Unable to connect to proxy', NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A8515E23F0>: Failed to resolve 'http' ([Errno 11001] getaddrinfo failed)"))). Trying the next proxy...
Searching for 'AETHER FINANCIAL SERVICES' using proxy: http//51.159.159.73:80
Proxy ['51.159.159.73:80', 'http'] failed with error: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=AETHER+FINANCIAL+SERVICES+official+company+website+-site%3Ainstagram.com+-site%3Alinkedin.com+-site%3Aannuaire-entreprises.data.gouv.fr&num=3&hl=en&

KeyboardInterrupt: 